Execute the following block of code by selecting it and clicking ``ctrl + enter`` to create an ``NvidiaRacecar`` class.  

In [1]:
from jetracer.nvidia_racecar import NvidiaRacecar

car = NvidiaRacecar()

96


In [2]:
import ipywidgets.widgets as widgets
from IPython.display import display

# create two sliders with range [-1.0, 1.0]
throttle_slider = widgets.FloatSlider(description='throttle', min=-1.0, max=1.0, step=0.01, orientation='vertical')
steering_slider = widgets.FloatSlider(description='steering', min=-1.0, max=1.0, step=0.01, orientation='horizontal')
steering_gain = widgets.BoundedFloatText(description='st_gain',min=-1.0, max=1.0, step=0.01, value=car.steering_gain)
steering_offset = widgets.BoundedFloatText(description='st_offset',min=-1.0, max=1.0, step=0.01, value=car.steering_offset)
throttle_gain = widgets.BoundedFloatText(description='th_gain',min=-1.0, max=1.0, step=0.01, value=car.throttle_gain)

# create a horizontal box container to place the sliders next to eachother
slider_container = widgets.HBox([throttle_slider, throttle_gain])
value_container = widgets.HBox([steering_slider,steering_gain,steering_offset])

# display the container in this cell's output
display(slider_container)
display(value_container)

In [3]:
import traitlets

throttle_link = traitlets.link((throttle_slider, 'value'), (car, 'throttle'))
steering_link = traitlets.link((steering_slider, 'value'), (car, 'steering'))
steering_gain_link = traitlets.link((steering_gain, 'value'), (car, 'steering_gain'))
throttle_gain_link = traitlets.link((throttle_gain, 'value'), (car, 'throttle_gain'))
steering_offset_link = traitlets.link((steering_offset, 'value'), (car, 'steering_offset'))

The ``NvidiaRacecar`` implements the ``Racecar`` class, so it has two attributes ``throttle`` and ``steering``. 

We can assign values in the range ``[-1, 1]`` to these attributes.  Execute the following to set the steering to 0.4.

> If the car does not respond, it may still be in ``manual`` mode.  Flip the manual override switch on the RC transmitter.

In [73]:
car.steering = 0

The ``NvidiaRacecar`` class has two values ``steering_gain`` and ``steering_bias`` that can be used to calibrate the steering.

We can view the default values by executing the cells below.

In [51]:
print(car.steering_gain)

-0.65


In [39]:
print(car.steering_offset)

0.0


The final steering value is computed using the equation

$y = a \times x + b$

Where,

* $a$ is ``car.steering_gain``
* $b$ is ``car.steering_offset``
* $x$ is ``car.steering``
* $y$ is the value written to the motor driver

You can adjust these values calibrate the car so that setting a value of ``0`` moves forward, and setting a value of ``1`` goes fully right, and ``-1`` fully left.

To set the throttle of the car to ``0.2``, you can call the following.

> Give JetRacer lots of space to move, and be ready on the manual override, JetRacer is *fast*

In [55]:
car.throttle = 0.1

The throttle also has a gain value that could be used to control the speed response.  The throttle output is computed as

$y = a \times x$

Where,

* $a$ is ``car.throttle_gain``
* $x$ is ``car.throttle``
* $y$ is the value written to the speed controller

Execute the following to print the default gain

In [38]:
print(car.throttle_gain)

1.0


Set the following to limit the throttle to half

In [53]:
car.throttle_gain = 1

Please note the throttle is directly mapped to the RC car.  When the car is stopped and a negative throttle is set, it will reverse.  If the car is moving forward and a negative throttle is set, it will brake.

That's it for this notebook!